# 1. Introduction to the MNIST data

For this classification project, we will be using the MNIST data set that contain 70,000 images of handwritten digits. In this data set, each row represents an image and each column a pixel from a 28 by 28 pixel image. The MNIST dataset is widely used to train classifiers and can be fetched using the helper function sklearn.datasets.fetch_openml.

The code below will download the data, and sample 20,000 rows from the original dataset. I will be reducing the number of rows in the dataset to decrease model size and to reduce build time for this project. The code below will also plot the first image in the data set which we can see is the number eight.

In [ ]:
from sklearn.datasets import fetch_openml
import matplotlib.pyplot as plt
import pandas as pd

# Load data
mnist = fetch_openml("mnist_784", version=1)

# Randomly sample 20000 rows from the original dataset
mnist_data = (
    mnist
    .data
    .sample(n=20000, random_state=42, axis=0, replace=False)
)

# Slice target by the same row sampling
target = (
    mnist
    .target
    .loc[mnist_data.index].astype('uint8')
)

# Reshape values to be 28x28
some_digit_image = (
    mnist_data
    .iloc[0]
    .values
    .reshape(28,28)
    .astype('float32')
)
plt.imshow(some_digit_image, cmap = "binary")
plt.axis("off")

<hr>

# 2. Training a K-Nearest Neighbors Classifier

First we will split the data into training and test set then train a K-nearest neighbour classifier using thescikit-learn library.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import  KNeighborsClassifier
from sklearn.model_selection import cross_val_score
import numpy as np

# Function to train KNN Classifier and show scores
def train_knn_model(features:np.array, target:np.array):

    # Train KNN Classifier
    knnclf = KNeighborsClassifier(weights='distance', n_neighbors=4)
    knnclf.fit(features, target)
    scores = cross_val_score(
        knnclf, features, target, scoring='accuracy', cv=10
    )
    print(f'Cross Validation Scores: {scores}')
    print(f'Average accuracy: {np.mean(scores)}')
    return knnclf, scores

# Split data to training and test set
train_features, test_features, train_target, test_target = train_test_split(
        mnist_data, target, test_size = 0.2, random_state = 42
)
knnclf, scores = train_knn_model(train_features, train_target)

The model achieves a decent average accuracy of 96% from cross validation. Let's evaluate the model's performance on the test_features data set and plot a confusion matrix with the show_cm function as shown below.

In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import accuracy_score

def show_cm(y_true, y_pred, labels):

    # Display Confusion matrix and show accuracy scores
    conf_mat = confusion_matrix(y_true, y_pred, labels=labels)
    disp = ConfusionMatrixDisplay(confusion_matrix=conf_mat, display_labels=labels)
    score = accuracy_score(y_true, y_pred)
    print(f'Accuracy: {score}')
    disp.plot();

# Make predictions
test_target_pred = knnclf.predict(test_features)
# Show confusion matrix
show_cm(test_target, test_target_pred, range(10))

Base on the accuracy on the test data set, we can see that our model fits the data. We get very similar prediction accuracy when comparing accuracies between the training and testing set. 

Furthermore, a confusion matrix, like above, is very effective in helping visualise the gaps in the model's performance. It will help us understand the kind of errors that the classifier is making.

The matrix indicates that there were 16 instances where the number 4 was misidentified for the number 9, and 12 instances where the number 8 was misidentified for the number 5. 

Looking at the images below, it is possible to see why some of these errors may occur as the number 4 and 9 do share some similar features. Likewise for the number 8 and 5.

In [ ]:
import matplotlib.pyplot as plt

def show_digits(pixel_vals):

    some_digit_image = (
        pixel_vals
        .values
        .reshape(28,28)
        .astype('float32')
    )
    plt.imshow(some_digit_image, cmap = "binary")
    plt.axis("off")

fours = train_features[train_target == 4]
nines = train_features[train_target == 9]
eights = train_features[train_target == 8]
fives = train_features[train_target == 5]

plt.figure(figsize=(8,8))
plt.subplot(221); show_digits(fours.iloc[0])
plt.subplot(222); show_digits(nines.iloc[0])
plt.subplot(223); show_digits(eights.iloc[1])
plt.subplot(224); show_digits(fives.iloc[1])

This insight is not going to affect model deployment on AWS but will help guide strategies to further improve the model. 
For now, we will save the model locally to be containerised as part of the lambda function using Docker.

In [ ]:
import joblib

joblib.dump(knnclf, 'app/knnclf.joblib')

<hr>

# 3. Initialising AWS S3 Bucket

The image below illustrates the overall resource infrastructure that will need to be deployed to support our lambda function. There are three key resources requirements for our application:

1. S3 Bucket to store data.
2. API gateway to manage HTTP requests.
3. Lambda function containing the predictive logic.

![](overview.png)

#### Serverless deployment of ML models

1. Test data is uploaded to a S3 bucket.

2. To initiate the lambda function, a POST HTTP request is sent through the Amazon API Gateway. 

3. Initialisation of the lambda function executes code that downloads the data from the S3 bucket and performs predictions. 

4. A HTTP response is returned to client with the predictions as a data payload.

The Lambda function will contain Python code that performs a prediction based on the test_features dataset stored on a S3 bucket. 

Therefore, we will first need to initialise a S3 bucket where we can host our data.

To do so, we will be interacting with AWS using the AWS Python SDK `boto3`. This package contains all the dependencies we require to integrate Python projects with AWS. 

Let's initialise a S3 bucket with the code below.

***Note:*** *The `bucket_name` has to be unique therefore you will have to replace the bucket_name with a name that is not taken.*

In [ ]:
import os
os.environ["AWS_ACCESS_KEY_ID"] = "<YOUR_KEY>"
os.environ["AWS_SECRET_ACCESS_KEY"] = "<YOUR_SECRET>"

In [ ]:
import boto3

def create_bucket(bucket_name:str) -> dict:

    s3 = boto3.client('s3')
    # The 'LocationConstraint' is unnecesary here since we want default region of 'us-east-1'
    # uncommenting this will throw an error
    # DO NOT create resources in other regions as you will not have permissions to delete them later
    response = s3.create_bucket(
        Bucket=bucket_name,
        # CreateBucketConfiguration={
        #     'LocationConstraint': 'us-east-1'
        # }
    )
    return response

# This name must be unique!
bucket_name = 'nm-lambda-demo-2024'
create_bucket(bucket_name)

#### Upload Test data to S3

The S3 bucket will host our test_features data set which we can call in our lambda function to perform a prediction.

To save an object currently in our workspace, we will be making use of BytesIO function from the io library. This will enable us to temporary store the test_features data set in a file object. This file object can be uploaded onto a S3 bucket by calling the .upload_fileobj function.

The bucket variable defines the destination S3 bucket and the key variable will define the file path in the bucket. The `bucket` and `key` variables will form part of the data payload in the POST HTTP request to our lambda function.


In [ ]:
from io import BytesIO
import joblib
import boto3

def UploadToS3(data, bucket:str, key:str):

    with BytesIO() as f:
        joblib.dump(data, f)
        f.seek(0)
        (
            boto3
            .client('s3')
            .upload_fileobj(Bucket=bucket, Key=key, Fileobj=f)
        )

key =  'validation/test_features.joblib'
UploadToS3(test_features, bucket_name, key)

#### List all objects in S3 bucket

We can check if the objects have been uploaded with the helper function below. list_s3_objects will list all objects in the defined bucket.

In [ ]:
import boto3

def listS3Objects(bucket:str) -> list:

     # Connect to s3 resource
    s3 = boto3.resource('s3')
    my_bucket = s3.Bucket(bucket)

    # List all object keys in s3 bucket
    obj_list = [object_summary.key for object_summary in my_bucket.objects.all()]
    return obj_list

listS3Objects(bucket_name)


We have now successfully initialised a S3 bucket to store the test_feature data. The next two key resources, API Gateway and lambda function, will be deployed using AWS Serverless Application Model (SAM). Refer to main blog for details.

<hr>

# 4. Deploying and Testing AWS Lambda Functions with SAM

#### 4.4. Building and testing the application locally.

AWS SAM provide functionality to build and locally test applications before deployment. 
Ensure docker is running. 

In a terminal window, navigate to the project directory and build the application in SAM.

a) If using Apple M Silicon, in `template_no_auth.yaml` line 25 change the value from `x86_64` to `arm64`

b) Ensure docker is running. In a terminal window, navigate to the project directory and build the application in SAM. Note, if having trouble with poetry lock file or install within the `Dockerfile`, re-download the `poetry.lock` file from [here](https://raw.githubusercontent.com/lloydhamilton/aws_lambda_no_authoriser/master/poetry.lock)

`sam build -t template_no_auth.yaml`

c) Locally deploy the dockerised lambda function.

`sam local start-api`

d) Locally invoke the function at http://127.0.0.1:3000/predict. Your URL may differ.

***Note***: *The `bucket` and `key` variable which references the test_feature data set on S3 will need to be passed as part of the data payload in the POST HTTP request.*

In [ ]:

knnclf = joblib.load('app/knnclf.joblib')

In [ ]:
import requests
import json
import numpy as np

data = {
    'bucket':bucket_name,
    'key':key,
}

headers = {
    'Content-type': "application/json"
}

# Main code for post HTTP request
url = "http://127.0.0.1:3000/predict"
# If you see the error, ClientError: An error occurred (403) when calling the HeadObject operation: Forbidden
# That means you need to `aws configure` your AWS key/secret
response = requests.request("POST", url, headers=headers, data=json.dumps(data))
lambda_predictions = np.array(response.json())
show_cm(test_target, lambda_predictions, range(10))

#### 4.5 Deploying on AWS Lambda

As easy as it was to deploy locally, SAM will also handle all the heavy lifting to deploy on AWS Lambda.

a) Build the application in SAM.

`sam build -t template_no_auth.yaml`

b) Deploy the application.

`sam deploy --guided`

Follow the prompts that guides you through the deployment configurations. Most of the settings I used were the default value with a few exceptions. See main blog for details.

c) Invoke your function by replacing the URL in the code below with the URL from the output out of a successfully deployed stack.

In [ ]:
test_features

In [ ]:
import matplotlib.pyplot as plt

def show_digits(pixel_vals):

    some_digit_image = (
        pixel_vals
        .values
        .reshape(28,28)
        .astype('float32')
    )
    plt.imshow(some_digit_image, cmap = "binary")
    plt.axis("off")

fours = train_features[train_target == 4]
nines = train_features[train_target == 9]
eights = train_features[train_target == 8]
fives = train_features[train_target == 5]

plt.figure(figsize=(8,8))
plt.subplot(221); show_digits(fours.iloc[0])
plt.subplot(222); show_digits(nines.iloc[0])
plt.subplot(223); show_digits(eights.iloc[1])
plt.subplot(224); show_digits(fives.iloc[1])

In [ ]:
import requests
import json
import numpy as np

data = {
    'bucket':bucket_name,
    'key':key,
}

headers = {
    'Content-type': "application/json"
}

# Main code for post HTTP request
# Update `url` with the value given after deployment
# NOTE: The first request may time out (due to long model loading?), but retry may be successful
url = "https://gvpqb9emm7.execute-api.us-east-1.amazonaws.com/dev/predict"
response = requests.request("POST", url, headers=headers, json=data)
lambda_predictions = np.array(response.json())
show_cm(test_target, lambda_predictions, range(10))

In [ ]:
# !jupyter nbconvert deploying_lambda.ipynb --to html --template classic